<a href="https://colab.research.google.com/github/akalpit23/Interpretable-ML/blob/main/Assignment-4/Assignment_04_Interpretable_ML_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIPI 590 - XAI | Assignment #04



## Akalpit Dawkhar

### **Interpretable ML-2**

### Overview

In [ ]:
pip install --upgrade skope-rules

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import lime
import sklearn
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, precision_recall_curve, roc_auc_score
from sklearn.datasets import load_diabetes
import six
from imodels import  RuleFitRegressor
from rulefit import RuleFit
from skrules import SkopeRules
from skrules import SkopeRules as SkopeRulesClassifier

/Users/akalpitdawkhar/Desktop/School/SEM 3/XAI/Interpretable-ML/xai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load the diabetes dataset
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = diabetes.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Convert to DataFrame
diabetes_df = pd.DataFrame(data=diabetes.data, columns=diabetes.feature_names)

# Print the first 5 rows
print(diabetes_df.head())

        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005670 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  
0 -0.002592  0.019907 -0.017646  
1 -0.039493 -0.068332 -0.092204  
2 -0.002592  0.002861 -0.025930  
3  0.034309  0.022688 -0.009362  
4 -0.002592 -0.031988 -0.046641  


In [ ]:
# Check the shape of your DataFrame
print(X_train.shape)

# Check the column names of your DataFrame
print(diabetes_df.columns)

(353, 10)
Index(['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], dtype='object')


In [ ]:
# Initialize the RuleFitRegressor
rf = RuleFitRegressor()

# Fit the model to your data
rf.fit(X_train, y_train)

RuleFitRegressor()

In [ ]:
# Use the model to make predictions
predictions = rf.predict(X_test)

In [ ]:
rule_df = rf.visualize()
rule_df

,rule,coef
3,bp,39.25
23,s5 > -0.03262,0.01
21,bmi <= 0.06924 and s5 <= 0.0199,-7.61
20,bmi <= 0.01319 and s4 <= 0.05313,-5.24
18,bmi <= 0.00942 and s5 <= 0.022,-5.04
11,bmi <= 0.00565 and s5 <= 0.0199,-8.32
16,bmi <= 0.00942 and s4 <= 0.03136,-9.78
17,bmi <= 0.07625 and s3 > -0.01579 and s5 <= 0.022,-2.45
19,bmi <= 0.00834 and s3 > -0.01579,-11.81
10,bmi <= 0.00511 and s2 <= 0.09106 and s5 <= 0.00579,-7.73


In [ ]:
# Calculate the mean absolute error
mae = mean_absolute_error(y_test, predictions)

# Calculate the mean squared error
mse = mean_squared_error(y_test, predictions)

# Calculate the R-squared score
r2 = r2_score(y_test, predictions)

# Print the metrics
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R2 Score: {r2}')

MAE: 44.44297419698701
MSE: 2965.867384017357
R2 Score: 0.44020716566909346


In [ ]:
# Get feature names
feature_names = diabetes.drop('Profit ($)', axis=1).columns.tolist()

# Initialize classifier
clf = SkopeRulesClassifier(max_depth_duplication=None,
                           n_estimators=30,
                           precision_min=0.2,
                           recall_min=0.01,
                           feature_names=feature_names)

for idx, species in enumerate(diabetes.target_names):
    X, y = diabetes.data, diabetes.target
    clf.fit(X, y == idx)
    rules = clf.rules_[0:3]
    print("Rules for iris", species)
    for rule in rules:
        print(rule)
    print()
    print(20*'=')
    print()

AttributeError: drop

# 6. Refrences:
